In [12]:
import utilities
import os
import socket

In [13]:
base_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic"
categories = utilities.get_directories_in_a_directory(base_dir)

### DNS Resolutions

In [ ]:
passed = ["Fashion-Style", "Sports", "ConnectedCar", "Wine-Beverages", "Pets-Animals", "Navigation-TripPlanners", 
          "Religion-Spirituality", "Dating", "Health-Fitness"]
for category in categories:
    print(category)
    dns_resolutions = {}
    category_name = category.split('/')[-1]
    if category_name in passed:
        continue
    network_json_dir = os.path.join(category, category_name+"-ENC-out.json")
    network_json = utilities.read_json(network_json_dir)
    
    for i in range(len(network_json)):
        if network_json[i]["_source"]["layers"]["frame"]["frame.protocols"] == 'eth:ethertype:ip:udp:dns':
            answers_key = []
            if "Answers" in network_json[i]["_source"]["layers"]["dns"].keys():
                answers_key  = network_json[i]["_source"]["layers"]["dns"]["Answers"].keys()
                for key in answers_key:
                    if "CNAME" in key:
                        #print("CNAME")
                        continue
                    dns_a = network_json[i]["_source"]["layers"]["dns"]["Answers"][key]["dns.a"]
                    dns_name = network_json[i]["_source"]["layers"]["dns"]["Answers"][key]["dns.resp.name"]
                    dns_resolutions[dns_a] = dns_name
    utilities.write_json(os.path.join(category, "dns_resolutions.json"), dns_resolutions)
                

### Aggregate DNS Resolutions

In [15]:
general_dns_resolutions = {}
for category in categories:
    print(category)
    category_name = category.split('/')[-1]
    dns_resultions = utilities.read_json(os.path.join(base_dir, category_name, "dns_resolutions.json"))
    for ip, dns_name in dns_resolutions.items():
        if ip not in general_dns_resolutions:
            general_dns_resolutions[ip] = dns_name
utilities.write_json(os.path.join(base_dir, "general_dns_resolutions.json"), general_dns_resolutions)

/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Fashion-Style
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Sports
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/ConnectedCar
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/SmartHome
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Wine-Beverages
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Pets-Animals
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Navigation-TripPlanners
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Religion-Spirituality
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Dating
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Health-Fitness


### Source & Destination IPs

In [3]:
passed = ["Fashion-Style", "Sports"]
for category in categories:
    print(category)
    ip = {"src": [], "dst": []}
    category_name = category.split('/')[-1]
    if category_name in passed:
        continue
    network_json_dir = os.path.join(category, category_name+"-ENC-out.json")
    network_json = utilities.read_json(network_json_dir)
    
    for i in range(len(network_json)):
        if 'ip' in network_json[i]["_source"]["layers"].keys():
            ip["src"].append(network_json[i]["_source"]["layers"]['ip']['ip.src'])
            ip["dst"].append(network_json[i]["_source"]["layers"]['ip']['ip.dst'])
    
    utilities.write_json(os.path.join(category, "ip.json"), ip)

/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Fashion-Style
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Sports
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/ConnectedCar
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/SmartHome
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Wine-Beverages
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Pets-Animals
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Navigation-TripPlanners
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Religion-Spirituality
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Dating
/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Traffic/Health-Fitness


# Skill Analysis

In [115]:
general_dns_resolutions = utilities.read_json(os.path.join(base_dir, "general_dns_resolutions.json"))

In [116]:
### Filter Lists
filter_list_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists"
disconnectme = os.path.join(filter_list_dir, "disconnectme_abp.txt")
moaab = os.path.join(filter_list_dir, "moaab_abp.txt")
piholeblocklist = os.path.join(filter_list_dir, "piholeblocklist_default_smarttv_abp.txt")

tracking_domains = set()
with open(disconnectme) as f:
    lines = f.readlines()
    for line in lines:
        current_line = line.split(',')
        #print(current_line)
        if current_line[0].startswith('||'):
            tracking_domains.add(current_line[0].split('||')[-1].replace("\n", ""))
print(len(tracking_domains))

with open(moaab) as f:
    lines = f.readlines()
    for line in lines:
        current_line = line.split(',')
        #print(current_line)
        if current_line[0].startswith('||'):
            tracking_domains.add(current_line[0].split('||')[-1].replace("\n", ""))
print(len(tracking_domains))

with open(piholeblocklist) as f:
    lines = f.readlines()
    for line in lines:
        current_line = line.split(',')
        #print(current_line)
        if current_line[0].startswith('||'):
            tracking_domains.add(current_line[0].split('||')[-1].replace("\n", ""))
print(len(tracking_domains))
utilities.write_json(os.path.join(filter_list_dir, "tracking_domains.json"), list(tracking_domains))

2653
86938
122046


In [117]:
all_non_resolvable_domains = set()

# Fashion-Style

In [118]:
ips = utilities.read_json(os.path.join(base_dir, "Fashion-Style", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'255.255.255.255', '239.255.255.250', '224.0.0.251'}
set()


In [119]:
all_non_resolvable_domains

{'224.0.0.251',
 '239.255.255.250',
 '255.255.255.255',
 '3.227.94.7',
 '44.199.138.71',
 '52.46.153.232',
 '52.94.233.250'}

### Tracking domains

In [120]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [121]:
print(non_resolvable_dst.union(non_resolvable_src))

{'255.255.255.255', '52.46.153.232', '224.0.0.251', '52.94.233.250', '3.227.94.7', '44.199.138.71', '239.255.255.250'}


# Connected Car

In [122]:
ips = utilities.read_json(os.path.join(base_dir, "ConnectedCar", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'255.255.255.255', '239.255.255.250', '224.0.0.251'}
set()


In [123]:
all_non_resolvable_domains

{'104.16.205.136',
 '13.226.239.25',
 '13.226.251.35',
 '13.226.251.56',
 '209.54.178.134',
 '224.0.0.251',
 '239.255.255.250',
 '255.255.255.255',
 '3.227.94.7',
 '34.237.111.59',
 '44.199.138.71',
 '44.199.52.214',
 '52.217.174.185',
 '52.45.237.36',
 '52.46.143.123',
 '52.46.153.232',
 '52.85.205.99',
 '52.94.227.146',
 '52.94.233.250',
 '52.94.243.11'}

### Tracking domains

In [124]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [125]:
print(non_resolvable_dst.union(non_resolvable_src))

{'52.85.205.99', '104.16.205.136', '224.0.0.251', '52.217.174.185', '44.199.52.214', '52.94.227.146', '52.46.143.123', '13.226.251.35', '52.45.237.36', '44.199.138.71', '52.94.243.11', '255.255.255.255', '13.226.239.25', '209.54.178.134', '13.226.251.56', '3.227.94.7', '34.237.111.59', '239.255.255.250'}


# Dating

In [126]:
ips = utilities.read_json(os.path.join(base_dir, "Dating", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
        all_non_resolvable_domains.add(ip)
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'224.0.0.251'}
set()


### Tracking domains

In [127]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [128]:
print(non_resolvable_dst.union(non_resolvable_src))

{'34.226.24.249', '52.119.197.96', '52.94.227.146', '52.46.143.123', '52.5.47.107', '224.0.0.251'}


# Health Fitness

In [129]:
ips = utilities.read_json(os.path.join(base_dir, "Health-Fitness", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'255.255.255.255', '239.255.255.250', '224.0.0.251'}
set()


### Tracking domains

In [130]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [131]:
print(non_resolvable_dst.union(non_resolvable_src))

{'52.217.37.68', '54.159.137.14', '52.119.196.87', '44.199.52.214', '205.251.243.55', '52.46.142.117', '255.255.255.255', '52.46.158.181', '52.217.135.184', '52.217.72.46', '3.227.94.7', '52.203.151.208', '224.0.0.251', '239.255.255.250', '52.46.138.235', '52.119.196.28', '52.119.198.106', '13.226.251.218', '52.46.146.248', '52.94.237.71', '176.32.101.122', '34.237.111.59'}


# Navigation TripPlanners

In [132]:
ips = utilities.read_json(os.path.join(base_dir, "Navigation-TripPlanners", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'255.255.255.255', '239.255.255.250', '224.0.0.251'}
set()


### Tracking domains

In [133]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [134]:
print(non_resolvable_dst.union(non_resolvable_src))

{'52.94.233.250', '44.199.52.214', '13.226.251.35', '52.119.198.101', '44.199.138.71', '52.94.243.11', '255.255.255.255', '52.46.158.181', '13.226.239.25', '3.227.94.7', '13.226.251.190', '224.0.0.251', '239.255.255.250', '52.45.237.36', '147.160.138.27', '13.226.251.218', '34.193.172.139', '52.94.242.236', '13.226.251.210', '205.185.216.42', '34.237.111.59'}


# Pets Animals

In [135]:
ips = utilities.read_json(os.path.join(base_dir, "Pets-Animals", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'255.255.255.255', '224.0.0.251', '147.160.138.27', '34.193.172.139', '239.255.255.250'}
set()


### Tracking domains

In [136]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [137]:
print(non_resolvable_dst.union(non_resolvable_src))

{'44.199.52.214', '52.95.149.165', '51.38.69.8', '44.199.138.71', '52.95.148.112', '52.94.243.11', '255.255.255.255', '34.226.24.249', '151.101.42.133', '103.89.74.186', '162.125.7.15', '52.217.128.160', '3.227.94.7', '89.187.187.19', '151.139.128.11', '239.255.255.250', '142.44.246.60', '151.101.198.133', '224.0.0.251', '52.95.142.21', '89.187.187.15', '52.216.130.101', '52.45.237.36', '89.187.187.12', '89.187.187.11', '54.231.40.130', '103.103.196.113', '147.160.138.27', '34.193.172.139', '52.119.198.186', '87.230.23.99', '52.95.150.153', '34.237.111.59', '52.216.244.222'}


# Religion Sprituality

In [139]:
ips = utilities.read_json(os.path.join(base_dir, "Religion-Spirituality", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'224.0.0.251'}
set()


### Tracking domains

In [140]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [141]:
print(non_resolvable_dst.union(non_resolvable_src))

{'13.107.246.69', '44.199.52.214', '63.243.143.155', '44.199.138.71', '209.54.180.27', '151.101.197.32', '34.226.24.249', '52.94.233.109', '107.155.101.124', '151.139.128.11', '224.0.0.251', '192.173.29.77', '52.119.197.96', '151.101.42.2', '54.239.18.66', '18.205.68.150', '208.80.55.212', '151.101.198.2', '151.101.41.32', '52.5.47.107'}


# Smart Homes

In [142]:
ips = utilities.read_json(os.path.join(base_dir, "SmartHome", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'255.255.255.255', '239.255.255.250', '224.0.0.251'}
set()


### Tracking domains

In [143]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [144]:
print(non_resolvable_dst.union(non_resolvable_src))

{'13.226.235.219', '13.226.251.85', '224.0.0.251', '52.10.255.225', '52.94.227.146', '44.199.138.71', '13.226.235.92', '13.226.251.159', '255.255.255.255', '13.226.239.25', '18.205.68.150', '52.94.242.236', '13.226.251.189', '52.94.240.157', '8.8.8.8', '13.226.251.190', '52.5.47.107', '239.255.255.250'}


# Wine Beverages

In [145]:
ips = utilities.read_json(os.path.join(base_dir, "Wine-Beverages", "ip.json"))


ip_src = set(ips['src'])
ip_dst = set(ips['dst'])

#What IPs only appeared in destination?
print(ip_dst-ip_src)

#What IPs only appeared in source?
print(ip_src-ip_dst)

# map IP to name ==> source
src_domains = set()
non_resolvable_src = set()
for ip in ip_src:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        src_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_src.add(ip)
        all_non_resolvable_domains.add(ip)
        
# map IP to name ==> destination
dst_domains = set()
non_resolvable_dst = set()
for ip in ip_dst:
    if ip.startswith("10.42.0"):
        continue
        #print(ip + " : " + "Local Device")
    if ip in general_dns_resolutions:
        #print(ip + " : " + general_dns_resolutions[ip])
        dst_domains.add(general_dns_resolutions[ip])
    else:
        #print(ip + " : " + "--")
        non_resolvable_dst.add(ip)
        all_non_resolvable_domains.add(ip)

{'224.0.0.251'}
set()


### Tracking domains

In [146]:
for src in src_domains:
    if src in tracking_domains:
        print(src)
for dst in dst_domains:
    if dst in tracking_domains:
        print(dst)

device-metrics-us-2.amazon.com
device-metrics-us-2.amazon.com


### non-resolvable domains

In [147]:
print(non_resolvable_dst.union(non_resolvable_src))

{'52.94.233.250', '52.10.255.225', '209.54.177.154', '44.199.52.214', '44.199.138.71', '52.94.243.11', '34.226.24.249', '52.46.153.232', '162.125.7.15', '18.205.68.150', '3.227.94.7', '34.237.111.59', '52.94.235.50', '224.0.0.251'}


# Non-resolvable IPs

In [149]:
all_non_resolvable_domains
utilities.write_json(os.path.join(filter_list_dir, "all_non_resolvable_domains.json"), list(all_non_resolvable_domains))

In [34]:
data_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data"
all_non_resolvable_domains = utilities.read_json(os.path.join(data_dir, "all_non_resolvable_domains.json"))
gethostbysocket = {}
all_non_resolvable_domains_new = set()

### Address other non-resolvable IPs

In [35]:
import socket
for ip in all_non_resolvable_domains:
    try:
        gethostbysocket[ip] = socket.gethostbyaddr(ip)[0]
    except:
        all_non_resolvable_domains_new.add(ip)
data_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data"        
utilities.write_json(os.path.join(data_dir, "all_non_resolvable_domains_new.json"), list(all_non_resolvable_domains_new))
utilities.write_json(os.path.join(data_dir, "other_dns_resolutions.json"), gethostbysocket)

In [36]:
filter_list_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/filter_lists"
tracking_domains = utilities.read_json(os.path.join(filter_list_dir, "tracking_domains.json"))
for ip, host in gethostbysocket.items():
    #print(ip, host)
    if host in tracking_domains:
        print(host)

In [37]:
#non_resolvable IPs
non_obs_ips = utilities.read_json(os.path.join(data_dir, "all_non_resolvable_domains_new.json"))
non_obs_ips

['54.239.18.66',
 '205.251.243.55',
 '52.94.243.11',
 '151.101.198.2',
 '52.94.233.250',
 '151.101.42.2',
 '52.46.132.115',
 '208.80.55.212',
 '52.94.227.146',
 '104.16.205.136',
 '103.103.196.113',
 '52.46.146.248',
 '151.101.198.133',
 '255.255.255.255',
 '13.107.246.69',
 '209.54.177.154',
 '52.46.138.235',
 '52.46.143.123',
 '209.54.178.134',
 '52.94.240.157',
 '52.94.233.109',
 '151.101.42.133',
 '52.46.142.117',
 '52.119.196.87',
 '162.125.7.15',
 '224.0.0.251',
 '103.89.74.186',
 '52.119.198.186',
 '151.139.128.11',
 '52.46.158.181',
 '52.94.237.71',
 '52.119.198.101',
 '63.243.143.155',
 '147.160.138.27',
 '192.173.29.77',
 '52.119.197.96',
 '52.94.242.236',
 '52.46.136.81',
 '52.119.196.28',
 '151.101.197.32',
 '176.32.101.122',
 '54.239.31.237',
 '151.101.41.32',
 '52.119.198.106',
 '209.54.180.27',
 '239.255.255.250',
 '52.46.153.232',
 '52.94.235.50']